In [1]:
#importing pandas and folium libraries
import pandas as pd
import folium

In [2]:
#reading 2019 data into respective dataframes
df1 = pd.read_csv('../Data/2019/Divvy_Trips_2019_Q1.csv')
df2 = pd.read_csv('../Data/2019/Divvy_Trips_2019_Q2.csv')
df3 = pd.read_csv('../Data/2019/Divvy_Trips_2019_Q3.csv')
df4 = pd.read_csv('../Data/2019/Divvy_Trips_2019_Q4.csv')

### The df2 has column names which are different than the others (df2, df3, df4 have similar column names). So renaming the columns as required

In [3]:
df2 = df2.rename(columns={"User Type":"usertype",
                          "03 - Rental Start Station Name":"from_station_name",
                          "02 - Rental End Station Name": "to_station_name",
                         "01 - Rental Details Local Start Time": "start_time",
                         "01 - Rental Details Local End Time": "end_time"})

In [4]:
#Concatenating the dataframes into a single dataframe
frames = [df1,df2,df3,df4]
dfbike = pd.concat(frames)
dfbike.reset_index(drop=True,inplace=True)
dfbike.head(5)

,trip_id,start_time,end_time,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear,01 - Rental Details Rental ID,01 - Rental Details Bike ID,01 - Rental Details Duration In Seconds Uncapped,03 - Rental Start Station ID,02 - Rental End Station ID,Member Gender,05 - Member Details Member Birthday Year
0,21742443.0,2019-01-01 00:04:37,2019-01-01 00:11:07,2167.0,390.0,199.0,Wabash Ave & Grand Ave,84.0,Milwaukee Ave & Grand Ave,Subscriber,Male,1989.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21742444.0,2019-01-01 00:08:13,2019-01-01 00:15:34,4386.0,441.0,44.0,State St & Randolph St,624.0,Dearborn St & Van Buren St (*),Subscriber,Female,1990.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21742445.0,2019-01-01 00:13:23,2019-01-01 00:27:12,1524.0,829.0,15.0,Racine Ave & 18th St,644.0,Western Ave & Fillmore St (*),Subscriber,Female,1994.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21742446.0,2019-01-01 00:13:45,2019-01-01 00:43:28,252.0,"1,783.0",123.0,California Ave & Milwaukee Ave,176.0,Clark St & Elm St,Subscriber,Male,1993.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21742447.0,2019-01-01 00:14:52,2019-01-01 00:20:56,1170.0,364.0,173.0,Mies van der Rohe Way & Chicago Ave,35.0,Streeter Dr & Grand Ave,Subscriber,Male,1994.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dfbike.columns

Index(['trip_id', 'start_time', 'end_time', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthyear',
       '01 - Rental Details Rental ID', '01 - Rental Details Bike ID',
       '01 - Rental Details Duration In Seconds Uncapped',
       '03 - Rental Start Station ID', '02 - Rental End Station ID',
       'Member Gender', '05 - Member Details Member Birthday Year'],
      dtype='object')

### Saving all the station names in a list "allSt19"

#### Comparing with 2018 data and only adding the stations which are new into a list "added19"
#### Also, checking for the stations that have been in 2018 but not in this year's data and adding it to "removed18"

In [6]:
allSt19 = list(dfbike['from_station_name'].unique())
len(allSt19)

640

In [7]:
allSt18 = ['May St & Taylor St',
 'Lake Shore Dr & Wellington Ave',
 'State St & Pearson St',
 'Morgan St & Polk St',
 'Damen Ave & Melrose Ave',
 'Ravenswood Ave & Irving Park Rd',
 'Damen Ave & Chicago Ave',
 'Damen Ave & Division St',
 'Blackstone Ave & Hyde Park Blvd',
 'Franklin St & Chicago Ave',
 'Benson Ave & Church St',
 'Mies van der Rohe Way & Chestnut St',
 'Kingsbury St & Erie St',
 'Rush St & Superior St',
 'Racine Ave & 18th St',
 'Daley Center Plaza',
 'Lincoln Ave & Diversey Pkwy',
 'Clark St & Elm St',
 'Eckhart Park',
 'Spaulding Ave & Armitage Ave',
 'Wilton Ave & Belmont Ave',
 'Clark St & Jarvis Ave',
 'State St & Kinzie St',
 'Ellis Ave & 60th St',
 'Western Ave & Division St',
 'Michigan Ave & Jackson Blvd',
 'Ashland Ave & Division St',
 'Southport Ave & Clybourn Ave',
 'Damen Ave & Charleston St',
 'Larrabee St & Webster Ave',
 'Damen Ave & Pierce Ave',
 'Green St & Madison St',
 'Sedgwick St & North Ave',
 'Kedzie Ave & Palmer Ct',
 'Clark St & Wrightwood Ave',
 'Sheridan Rd & Irving Park Rd',
 'Drake Ave & Fullerton Ave',
 'Clark St & Schiller St',
 'Dearborn Pkwy & Delaware Pl',
 'Wells St & Concord Ln',
 'Milwaukee Ave & Grand Ave',
 'Halsted St & Willow St',
 'Racine Ave (May St) & Fulton St',
 'Cottage Grove Ave & 71st St',
 'California Ave & Milwaukee Ave',
 'Southport Ave & Waveland Ave',
 'Broadway & Belmont Ave',
 'Clark St & Lake St',
 'Wentworth Ave & 35th St',
 'Wilton Ave & Diversey Pkwy',
 'Halsted St & Wrightwood Ave',
 'Glenwood Ave & Morse Ave',
 'Damen Ave & Cullerton St',
 'Clark St & Grace St',
 'Lincoln Ave & Fullerton Ave',
 'Greenview Ave & Jarvis Ave',
 '2112 W Peterson Ave',
 'Green St & Randolph St',
 'Kedzie Ave & Leland Ave',
 'Ogden Ave & Chicago Ave',
 'Orleans St & Chestnut St (NEXT Apts)',
 'Halsted St & Roscoe St',
 'Wells St & Huron St',
 'Michigan Ave & Madison St',
 'Broadway & Cornelia Ave',
 'Michigan Ave & Washington St',
 'Clark St & Elmdale Ave',
 'Lake Shore Dr & Belmont Ave',
 'Lincoln Ave & Waveland Ave',
 'Lake Park Ave & 53rd St',
 'McClurg Ct & Illinois St',
 'Greenview Ave & Fullerton Ave',
 'Larrabee St & Oak St',
 'Leavitt St & Archer Ave',
 'Jefferson St & Monroe St',
 'Ashland Ave & Grand Ave',
 'Drake Ave & Montrose Ave',
 'Franklin St & Lake St',
 'Wood St & Taylor St',
 'Franklin St & Jackson Blvd',
 'Cottage Grove Ave & Oakwood Blvd',
 'Morgan St & 18th St',
 'Shore Dr & 55th St',
 'California Ave & Montrose Ave',
 'Paulina St & 18th St',
 'Dearborn St & Van Buren St (*)',
 'Clinton St & Tilden St',
 'Sheffield Ave & Fullerton Ave',
 'Racine Ave & 15th St',
 'Wabash Ave & Wacker Pl',
 'Rush St & Cedar St',
 'MLK Jr Dr & Pershing Rd',
 'Clark St & 9th St (AMLI)',
 'Orleans St & Merchandise Mart Plaza',
 'Field Blvd & South Water St',
 'Dayton St & North Ave',
 'Clinton St & Lake St',
 'Washtenaw Ave & Lawrence Ave',
 'Desplaines St & Kinzie St',
 'Dearborn St & Erie St',
 'Clifton Ave & Armitage Ave',
 'Clinton St & Madison St',
 'State St & Randolph St',
 'Lincoln Ave & Leavitt St',
 'Canal St & Adams St',
 'Clark St & Chicago Ave',
 'Lincoln Ave & Belmont Ave',
 'Clark St & Drummond Pl',
 'Lake Shore Dr & Diversey Pkwy',
 'Wentworth Ave & 33rd St',
 'Honore St & Division St',
 'Streeter Dr & Grand Ave',
 'Sedgwick St & Huron St',
 'Carpenter St & Huron St',
 'Halsted St & Maxwell St',
 'Pine Grove Ave & Waveland Ave',
 'Clybourn Ave & Division St',
 'Indiana Ave & Roosevelt Rd',
 'Sheffield Ave & Wellington Ave',
 'LaSalle Dr & Huron St (*)',
 'Ellis Ave & 58th St',
 'Federal St & Polk St',
 'Humboldt Blvd & Armitage Ave',
 'Kingsbury St & Kinzie St',
 'Pine Grove Ave & Irving Park Rd',
 'Clark St & Wellington Ave',
 'Aberdeen St & Monroe St',
 'Sheffield Ave & Webster Ave',
 'Broadway & Argyle St',
 'Montrose Harbor',
 'Clark St & Bryn Mawr Ave',
 'Michigan Ave & 18th St',
 'May St & Cullerton St',
 'Bissell St & Armitage Ave',
 'Morgan St & 31st St',
 'Desplaines St & Randolph St',
 'Sheridan Rd & Buena Ave',
 'Columbus Dr & Randolph St',
 'Dearborn St & Monroe St',
 'Broadway & Thorndale Ave',
 'Stony Island Ave & 71st St',
 'Emerald Ave & 28th St',
 'Southport Ave & Roscoe St',
 'Halsted St & 18th St',
 'McCormick Place',
 'Broadway & Waveland Ave',
 'Wabash Ave & Grand Ave',
 'Clarendon Ave & Gordon Ter',
 'Sedgwick St & Webster Ave',
 'Canal St & Taylor St',
 'MLK Jr Dr & 29th St',
 'Cityfront Plaza Dr & Pioneer Ct',
 'Michigan Ave & Oak St',
 'Wells St & Elm St',
 'Stetson Ave & South Water St',
 'Lake Shore Dr & Ohio St',
 'Lake Shore Dr & North Blvd',
 'Seeley Ave & Roscoe St',
 'Lincoln Ave & Roscoe St',
 'Theater on the Lake',
 'St. Clair St & Erie St',
 'Clark St & Armitage Ave',
 'Sheffield Ave & Willow St',
 'Broadway & Barry Ave',
 'Larrabee St & Menomonee St',
 'Wabash Ave & Roosevelt Rd',
 'Clinton St & Polk St (*)',
 'Damen Ave & Leland Ave',
 'Sheffield Ave & Waveland Ave',
 'Stockton Dr & Wrightwood Ave',
 'Desplaines St & Jackson Blvd',
 'Lakeview Ave & Fullerton Pkwy',
 'Milwaukee Ave & Rockwell St',
 'Southport Ave & Irving Park Rd',
 'Richmond St & Diversey Ave',
 'Wood St & Hubbard St',
 'Ada St & Washington Blvd',
 'Halsted St & 47th Pl',
 'Aberdeen St & Jackson Blvd',
 'Wells St & Walton St',
 'Noble St & Milwaukee Ave',
 'Broadway & Berwyn Ave',
 'Clark St & Winnemac Ave (Temp)',
 'Adler Planetarium',
 'Shields Ave & 28th Pl',
 'Shields Ave & 31st St',
 'Michigan Ave & 8th St',
 'Ellis Ave & 53rd St',
 'Larrabee St & Armitage Ave',
 'Southport Ave & Clark St',
 'Eastlake Ter & Rogers Ave',
 'Loomis St & Jackson Blvd',
 'LaSalle St & Washington St',
 'Ridge Blvd & Howard St',
 'Michigan Ave & 14th St',
 'Halsted St & Dickens Ave',
 'Wabash Ave & 16th St',
 'Oakley Ave & Irving Park Rd',
 'Racine Ave & Randolph St',
 'Wells St & Evergreen Ave',
 'Campbell Ave & Fullerton Ave',
 'Woodlawn Ave & 55th St',
 'Morgan St & Lake St',
 'Burling St (Halsted) & Diversey Pkwy (Temp)',
 'Elmwood Ave & Austin St',
 'Campbell Ave & North Ave',
 'Franklin St & Monroe St',
 'Leavitt St & Armitage Ave',
 'Ashland Ave & Chicago Ave',
 'Kosciuszko Park',
 'Kedzie Ave & Milwaukee Ave',
 'South Shore Dr & 74th St',
 'Loomis St & Lexington St',
 'California Ave & Division St',
 'Wabash Ave & 9th St',
 'Milwaukee Ave & Wabansia Ave',
 'Ravenswood Ave & Lawrence Ave',
 'Leavitt St & Lawrence Ave',
 'Ridge Blvd & Touhy Ave',
 'Shedd Aquarium',
 'Paulina Ave & North Ave',
 'Burnham Harbor',
 'LaSalle St & Illinois St',
 'Ashland Ave & Augusta Blvd',
 'Ogden Ave & Race Ave',
 'Paulina St & Montrose Ave',
 'Lincoln Ave & Belle Plaine Ave',
 'Sheffield Ave & Kingsbury St',
 'Wood St & Milwaukee Ave',
 'Damen Ave & Thomas St (Augusta Blvd)',
 'Ashland Ave & Blackhawk St',
 'Central St & Girard Ave',
 'Troy St & North Ave',
 'Damen Ave & Foster Ave',
 'Ashland Ave & Wrightwood Ave',
 'Ashland Ave & Harrison St',
 'Marshfield Ave & Cortland St',
 '900 W Harrison St',
 'Blue Island Ave & 18th St',
 'Clark St & Lincoln Ave',
 'California Ave & Byron St',
 'Lakefront Trail & Bryn Mawr Ave',
 'Wolcott Ave & Polk St',
 'Millennium Park',
 'Wentworth Ave & 24th St',
 'Michigan Ave & Congress Pkwy',
 'Wells St & Hubbard St',
 'Halsted St & Polk St',
 'Sheridan Rd & Noyes St (NU)',
 'Wells St & 19th St',
 'California Ave & Fletcher St',
 'Clinton St & 18th St',
 'Rockwell St & Eastwood Ave',
 'Southport Ave & Wellington Ave',
 'State St & Van Buren St',
 'Western Ave & Leland Ave',
 'Dusable Harbor',
 'Clark St & Congress Pkwy',
 'Museum of Science and Industry',
 'Canal St & Jackson Blvd',
 'Lake Shore Dr & Monroe St',
 'Wentworth Ave & Cermak Rd (Temp)',
 'Kimbark Ave & 53rd St',
 'Avers Ave & Belmont Ave',
 'California Ave & Altgeld St',
 'Clark St & Berwyn Ave',
 'Western Ave & Walton St',
 'Fairbanks Ct & Grand Ave',
 'Leavitt St & North Ave',
 'Clark St & North Ave',
 'Calumet Ave & 18th St',
 'Western Ave & Lunt Ave',
 'Maplewood Ave & Peterson Ave',
 'McClurg Ct & Erie St',
 'Greenview Ave & Diversey Pkwy',
 'Ellis Ave & 55th St',
 'Southport Ave & Belmont Ave',
 'Wabash Ave & Adams St',
 'Damen Ave & Cortland St',
 'Broadway & Ridge Ave',
 'Ravenswood Ave & Berteau Ave',
 'Larrabee St & Kingsbury St',
 'California Ave & Cortez St',
 'Troy St & Elston Ave',
 'Peoria St & Jackson Blvd',
 'Sheffield Ave & Wrightwood Ave',
 'Union Ave & Root St',
 'Claremont Ave & Hirsch St',
 'Wells St & Polk St',
 'State St & Harrison St',
 'Damen Ave & Grand Ave',
 'Loomis St & Taylor St (*)',
 'Canal St & Harrison St',
 'Central Park Ave & North Ave',
 'Clark St & Lunt Ave',
 'Western Ave & Roscoe St',
 'Lake Park Ave & 56th St',
 'State St & 29th St',
 'Canal St & Madison St',
 'Kedzie Ave & Lake St',
 'Cannon Dr & Fullerton Ave',
 'Logan Blvd & Elston Ave',
 'Halsted St & 63rd St',
 'Wacker Dr & Washington St',
 'Kedzie Ave & Bryn Mawr Ave',
 'Greenwood Ave & 47th St',
 'Buckingham Fountain',
 'MLK Jr Dr & 83rd St',
 'Halsted St & Clybourn Ave (*)',
 'Warren Park West',
 'Orleans St & Elm St (*)',
 'LaSalle St & Jackson Blvd',
 'Racine Ave & Fullerton Ave',
 'Indiana Ave & 40th St',
 'Fort Dearborn Dr & 31st St',
 'Ashland Ave & Wellington Ave',
 'Michigan Ave & Lake St',
 'Marine Dr & Ainslie St',
 'Sheridan Rd & Greenleaf Ave',
 'Harper Ave & 59th St',
 'Field Museum',
 'Ashland Ave & Grace St',
 'Michigan Ave & Pearson St',
 'Sheridan Rd & Montrose Ave',
 'Mies van der Rohe Way & Chicago Ave',
 'Cottage Grove Ave & 47th St',
 'Francisco Ave & Foster Ave',
 'Emerald Ave & 31st St',
 'Larrabee St & Division St',
 'Christiana Ave & Lawrence Ave',
 'Drake Ave & Addison St',
 'Chicago Ave & Dempster St',
 'Broadway & Sheridan Rd',
 'Chicago Ave & Washington St',
 'Franklin St & Quincy St',
 'Broadway & Granville Ave',
 'Halsted St & North Branch St',
 'Dearborn St & Adams St',
 'California Ave & Francis Pl',
 'Lincoln Ave & Winona St',
 'Calumet Ave & 21st St',
 'Millard Ave & 26th St',
 'Broadway & Wilson Ave',
 'Halsted St & 35th St (*)',
 'Winchester (Ravenswood) Ave & Balmoral Ave',
 'Cornell Ave & Hyde Park Blvd',
 'Winchester Ave & Elston Ave',
 'South Shore Dr & 67th St',
 'Racine Ave & Wrightwood Ave',
 'Central St Metra',
 'Talman Ave & Addison St',
 'Clark St & Leland Ave',
 'Clark St & Columbia Ave',
 'Racine Ave & Congress Pkwy',
 'Morgan Ave & 14th Pl',
 'Clark St & Randolph St',
 'Central Park Ave & Elbridge Ave',
 'Rush St & Hubbard St',
 'University Ave & 57th St',
 'Central Ave & Harrison St',
 'California Ave & 21st St',
 'Indiana Ave & 26th St',
 'Ashland Ave & Lake St (Temp)',
 'California Ave & 23rd Pl',
 'Clinton St & Roosevelt Rd',
 'Western Ave & Winnebago Ave',
 'Clark St & Schreiber Ave',
 'Racine Ave & 13th St',
 'Larrabee St & North Ave',
 'Lake Park Ave & 35th St',
 'Paulina St & Howard St',
 'Clarendon Ave & Leland Ave',
 'Calumet Ave & 71st St',
 'Canal St & Monroe St (*)',
 'Valli Produce - Evanston Plaza',
 'Halsted St & Roosevelt Rd',
 'State St & 35th St',
 'Elston Ave & Wabansia Ave',
 'Damen Ave & Madison St',
 'Kedzie Ave & Chicago Ave',
 'Sangamon St & Washington Blvd (*)',
 'Artesian Ave & Hubbard St',
 'Warren Park East',
 'Calumet Ave & 33rd St',
 'Lincoln Ave & Addison St',
 'Clinton St & Washington Blvd',
 'Sheridan Rd & Lawrence Ave',
 'Clark St & Montrose Ave',
 'Indiana Ave & 31st St',
 'Delano Ct & Roosevelt Rd',
 'Winthrop Ave & Lawrence Ave',
 'Damen Ave & Sunnyside Ave',
 'Wabash Ave & Cermak Rd',
 'Calumet Ave & 51st St',
 'California Ave & Lake St',
 'Clarendon Ave & Junior Ter',
 'Ritchie Ct & Banks St',
 'Western Ave & 21st St',
 'State St & 33rd St',
 'Sheridan Rd & Loyola Ave',
 'University Library (NU)',
 'Manor Ave & Leland Ave',
 'Ogden Ave & Roosevelt Rd',
 'California Ave & North Ave',
 'Sedgwick St & Schiller St',
 'Halsted St & Archer Ave',
 'Sawyer Ave & Irving Park Rd',
 'Wood St & 35th St',
 'Racine Ave & Belmont Ave',
 'Normal Ave & Archer Ave',
 'Cottage Grove Ave & 67th St',
 'Hermitage Ave & Polk St',
 'Princeton Ave & 47th St',
 'Central Ave & Chicago Ave',
 'Woodlawn Ave & Lake Park Ave',
 'Loomis St & Archer Ave',
 'Monticello Ave & Irving Park Rd',
 'Wallace St & 35th St',
 'Ashland Ave & Belle Plaine Ave',
 'Albany Ave & Bloomingdale Ave',
 'California Ave & 26th St',
 'Dorchester Ave & 63rd St',
 'Leavitt St & Addison St',
 'Southport Ave & Wrightwood Ave',
 'Central Park Ave & 24th St',
 'Kedzie Ave & 21st St',
 'Princeton Ave & Garfield Blvd',
 'Glenwood Ave & Touhy Ave',
 'State St & 19th St',
 'Prairie Ave & 43rd St',
 'Campbell Ave & Montrose Ave',
 'Stony Island Ave & 67th St',
 'Fairfield Ave & Roosevelt Rd',
 'Bosworth Ave & Howard St',
 'Western Ave & Congress Pkwy',
 'Chicago Ave & Sheridan Rd',
 'Cottage Grove Ave & 63rd St',
 'Prairie Ave & Garfield Blvd',
 'Ashland Ave & 21st St',
 'Bennett Ave & 79th St',
 'Halsted St & 37th St',
 'MLK Jr Dr & 47th St',
 'St. Louis Ave & Balmoral Ave',
 'MLK Jr Dr & 63rd St',
 'Kedzie Ave & Foster Ave',
 'LaSalle St & Adams St',
 'Western Ave & Granville Ave',
 'Ashland Ave & 13th St',
 'Stony Island Ave & 64th St',
 'Keystone Ave & Montrose Ave',
 '63rd St Beach',
 'Damen Ave & Clybourn Ave',
 'Wolcott (Ravenswood) Ave & Montrose Ave (*)',
 'Oakley Ave & Touhy Ave',
 'Halsted St & 21st St',
 'Knox Ave & Montrose Ave',
 'Evans Ave & 75th St',
 'Ogden Ave & Congress Pkwy',
 'Clark St & Touhy Ave',
 'Sacramento Blvd & Franklin Blvd',
 'Damen Ave & Wellington Ave',
 'Austin Blvd & Lake St',
 'Keystone Ave & Fullerton Ave',
 'State St & 79th St',
 'Stony Island Ave & 75th St',
 'Pulaski Rd & Lake St',
 'Albany Ave & Montrose Ave',
 'Conservatory Dr & Lake St',
 'Western Ave & 28th St',
 'Damen Ave & Coulter St',
 'Dodge Ave & Church St',
 'Dorchester Ave & 49th St',
 'Jeffery Blvd & 67th St',
 'Albany Ave & 26th St',
 'Wolcott Ave & Fargo Ave',
 'Washtenaw Ave & Ogden Ave (*)',
 'Cottage Grove Ave & 51st St',
 'DIVVY Map Frame B/C Station',
 'Avondale Ave & Irving Park Rd',
 'Kilbourn Ave & Milwaukee Ave',
 'Kimball Ave & Belmont Ave',
 'Calumet Ave & 35th St',
 'Budlong Woods Library',
 'Lake Park Ave & 47th St',
 'Western Ave & Monroe St',
 'Ashland Ave & Archer Ave',
 'South Shore Dr & 71st St',
 'Central Park Ave & Ogden Ave',
 'Stave St & Armitage Ave',
 'Kilbourn Ave & Irving Park Rd',
 'Western Ave & Howard St',
 'Saginaw Ave & Exchange Ave',
 'Spaulding Ave & Division St',
 'DuSable Museum',
 'Rhodes Ave & 32nd St',
 'Ashland Ave & McDowell Ave',
 'Jeffery Blvd & 71st St',
 'MLK Jr Dr & 56th St (*)',
 'Racine Ave & Garfield Blvd',
 'Austin Blvd & Madison St',
 'Eberhart (Vernon) Ave & 79th St',
 'Greenwood Ave & 79th St',
 'Stony Island Ave & South Chicago Ave',
 'Woodlawn Ave & 75th St',
 'Halsted St & 56th St',
 'Racine Ave & 35th St',
 'Western Ave & 24th St',
 'Cottage Grove Ave & 43rd St',
 'State St & Pershing Rd',
 'Karlov Ave & Madison St',
 'Damen Ave & Pershing Rd',
 'Vernon Ave & 75th St',
 'Halsted St & 51st St',
 'Yates Blvd & 75th St',
 'Exchange Ave & 79th St',
 'Austin Blvd & Chicago Ave',
 'Phillips Ave & 79th St',
 'Kostner Ave & Lake St',
 'Kedzie Ave & 24th St',
 'Wabash Ave & 83rd St',
 'Ashland Ave & Pershing Rd',
 'Throop St & 52nd St',
 'State St & 76th St',
 'Stony Island Ave & 82nd St',
 'Wentworth Ave & 63rd St',
 'Central Ave & Lake St',
 'Morgan St & Pershing Rd',
 'Wabash Ave & 87th St',
 'Pulaski Rd & Eddy St (Temp)',
 'Elizabeth St & 47th St',
 'Eberhart Ave & 61st St',
 'Kedzie Ave & Roosevelt Rd',
 'Halsted St & 59th St',
 'Ellis Ave & 83rd St',
 'South Chicago Ave & 83rd St',
 'Cottage Grove Ave & 83rd St',
 'Ashland Ave & 63rd St',
 'Kedzie Ave & Harrison St',
 'Milwaukee Ave & Cuyler Ave',
 'Ashland Ave & 50th St',
 'Hoyne Ave & 47th St',
 'Jeffery Blvd & 76th St',
 'Marshfield Ave & 44th St',
 'Rainbow Beach',
 'Western Blvd & 48th Pl',
 'Perry Ave & 69th St',
 'Halsted St & 69th St',
 'Damen Ave & 51st St',
 'Pulaski Rd & Congress Pkwy',
 'Financial Pl & Congress Pkwy (Temp)',
 'Central Park Blvd & 5th Ave',
 'Cicero Ave & Lake St',
 'Kenton Ave & Madison St',
 'Laramie Ave & Kinzie St',
 'Aberdeen St & Randolph St',
 'Marshfield Ave & 59th St',
 'Racine Ave & 61st St',
 'Normal Ave & 72nd St',
 'Kostner Ave & Adams St',
 'Cottage Grove Ave & 78th St',
 'Ashland Ave & 69th St',
 'Cicero Ave & Quincy St',
 'Laramie Ave & Gladys Ave',
 'Central Ave & Madison St',
 'May St & 69th St',
 'Ashland Ave & Garfield Blvd',
 'Damen Ave & 59th St',
 'Seeley Ave & Garfield Blvd',
 'Shields Ave & 43rd St',
 'Cicero Ave & Flournoy St',
 'DIVVY CASSETTE REPAIR MOBILE STATION',
 'Racine Ave & 65th St',
 'Laramie Ave & Madison St',
 'Commercial Ave & 83rd St',
 'Ashland Ave & 66th St',
 'Phillips Ave & 83rd St',
 'Lincoln Ave & Sunnyside Ave',
 'Marion St & South Blvd',
 'Malcolm X College',
 'Kildare Ave & Montrose Ave',
 'Leavitt St & Chicago Ave',
 'Wood St & Augusta Blvd',
 'Walsh Park',
 'Lakefront Trail & Wilson Ave',
 'Orleans St & Hubbard St (*)',
 'Fairbanks St & Superior St (*)',
 'Clinton St & Jackson Blvd (*)',
 'Wood St & Chicago Ave (*)',
 'Smith Park (*)',
 'State St & 54th St',
 'Central Park Ave & Bloomingdale Ave',
 'Hoyne Ave & Balmoral Ave',
 'Sheridan Rd & Columbia Ave',
 'Western Ave & Fillmore St (*)',
 'Bernard St & Elston Ave',
 'Racine Ave & Washington Blvd (*)',
 'Stewart Ave & 63rd St (*)',
 'Eggleston Ave & 69th St (*)',
 'Damen Ave & Walnut (Lake) St (*)',
 'Leavitt St & Division St (*)',
 'Carpenter St & 63rd St',
 'Dodge Ave & Mulford St',
 'Evanston Civic Center',
 'Lincolnwood Dr & Central St',
 'Clark St & Newport St',
 'Michigan Ave & 71st St',
 'Leavitt St & Belmont Ave (*)',
 'Cornell Dr & Hayes Dr']

In [8]:
len(allSt18)

608

In [9]:
removed18 = list(set(allSt18) - set(allSt19))
len(removed18)

10

In [10]:
added19 = list(set(allSt19) - set(allSt18))
len(added19)

42

In [27]:
added19

['Financial Pl & Ida B Wells Dr',
 'MLK Jr Dr & 56th St',
 'Clark St & Ida B Wells Dr',
 'Halsted St & Clybourn Ave',
 'Paulina St & Flournoy St',
 'Archer (Damen) Ave & 37th St',
 'Special Events',
 'HUBBARD ST BIKE CHECKING (LBS-WH-TEST)',
 'Washtenaw Ave & Ogden Ave',
 'Wolcott (Ravenswood) Ave & Montrose Ave',
 'LaSalle Dr & Huron St',
 'California Ave & Francis Pl (Temp)',
 'Orleans St & Elm St',
 'Latrobe Ave & Chicago Ave',
 'Financial Pl & Ida B Wells Dr (Temp)',
 'Vernon Ave & 79th St',
 'Ashland Ave & Lake St',
 'LBS - BBB La Magie',
 'Franklin St & Adams St (Temp)',
 'Wentworth Ave & 24th St (Temp)',
 'Fairbanks St & Superior St',
 'Clinton St & Jackson Blvd',
 'Buckingham Fountain (Temp)',
 'Throop (Loomis) St & Taylor St',
 'Michigan Ave & Ida B Wells Dr',
 'Halsted St & 35th St',
 'Franklin St & Chicago Ave (Temp)',
 'Dearborn St & Van Buren St',
 'Franklin St & Illinois St',
 'Sangamon St & Washington Blvd',
 'Clark St & Winnemac Ave',
 'Canal St & Monroe St',
 'Orleans 

# Plotting all added stations of 2019 and removed stations of 2018

In [11]:
#reading the 'final_stations.csv' into a dataframe
#using all the stations co-ordinates and comparing with each of the year's stations added/ removed list and adding locations 
#     to each of the list to plot yearly data.

final = pd.read_csv('final_stations.csv')

In [12]:
final.head(1)

,Unnamed: 0,Station,Latitude,Longitude
0,0,Ogden Ave & Race Ave,41.891795,-87.658751


In [13]:
#Removing the unnecessary column in the dataframe
final = final.drop('Unnamed: 0', axis =1)

In [14]:
final.head(1)

,Station,Latitude,Longitude
0,Ogden Ave & Race Ave,41.891795,-87.658751


In [15]:
#Converting the dataframe into a dictionary
# key : Station name and value: latitude and longitude

locations = dict()

for station in final['Station']:
    locations[station] = {'lat': final[final['Station'] == station]['Latitude'].to_list()[0], 'long': final[final['Station'] == station]['Longitude'].to_list()[0]}

#printing out the first item in the dictionary to check the key-value pair
list(locations.items())[0]

('Ogden Ave & Race Ave', {'lat': 41.891795, 'long': -87.658751})

In [16]:
#Adding co-ordinates to all the stations that are added this year
n19 = {k: locations[k] for k in locations.keys() & set(added19)}

#printing out the first item in the dictionary to check the key-value pair
list(n19.items())[0]

('Financial Pl & Ida B Wells Dr',
 {'lat': 41.875023628, 'long': -87.63309404299999})

In [17]:
#Converting the dictionary into dataframe
st19 = pd.DataFrame.from_dict(n19, orient='index')

In [18]:
#Resetting the index
st19.reset_index(inplace=True)

#Renaming the index column to Station name
st19 = st19.rename(columns = {'index':'Station Name'})

In [19]:
#Adding a column with values of color for every station
st19['color'] = st19.apply(lambda row: 'lightgray', axis=1)

#printing to check the structure of the dataframe
st19.head(1)

,Station Name,lat,long,color
0,Financial Pl & Ida B Wells Dr,41.875024,-87.633094,lightgray


In [20]:
removed18 = ['Clark St & Congress Pkwy',
 'Ashland Ave & Harrison St',
 'Financial Pl & Congress Pkwy (Temp)',
 'Normal Ave & 72nd St',
 'Clark St & Winnemac Ave (Temp)',
 'DIVVY CASSETTE REPAIR MOBILE STATION',
 'Lincoln Ave & Leavitt St',
 'Marion St & South Blvd',
 'Saginaw Ave & Exchange Ave',
 'Michigan Ave & Congress Pkwy',
 'Eberhart (Vernon) Ave & 79th St']

In [21]:
#adding co-ordinates for the removed stations in 2018
rm18 = {k: locations[k] for k in locations.keys() & set(removed18)}

#printing out the first item in the dictionary to check the key-value pair
list(rm18.items())[0]

('Lincoln Ave & Leavitt St', {'lat': 41.962391, 'long': -87.684146})

In [22]:
#Converting the dictionary into dataframe
rm18 = pd.DataFrame.from_dict(rm18, orient='index')

In [23]:
#Resetting the index
rm18.reset_index(inplace=True)

#Renaming the index column to Station name
rm18 = rm18.rename(columns = {'index':'Station Name'})

In [24]:
#Adding a column with values of color for every station
rm18['color'] = rm18.apply(lambda row: 'white', axis=1)

#printing to check the structure of the dataframe
rm18.head(1)

,Station Name,lat,long,color
0,Lincoln Ave & Leavitt St,41.962391,-87.684146,white


In [25]:
#merging both the added19 and removed18 dataframes
merged = pd.concat([rm18, st19], axis =0)

In [26]:
#setting marker as the color mentioned in the color column of the dataframe
marker = merged.loc[merged['color']!='']

map = folium.Map(location=[marker.lat.mean(), marker.long.mean()], zoom_start=12,
                     width=980,height=590, control_scale=True, tiles='OpenStreetMap')
marker.apply(lambda row:folium.Marker(location=[row["lat"], row["long"]],
        popup=row['Station Name'],
          icon=folium.Icon(color=row['color'])
            ).add_to(map), axis=1)

map